In [1]:
import os
import re
import json
import traceback
import huggingface_hub

from tqdm import tqdm
from openai import OpenAI

from kiwipiepy import Kiwi ## 한글 형태소 분석기

from langchain.schema import Document
from langchain.vectorstores import FAISS  ## 벡터 DB
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import JSONLoader
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv
load_dotenv("../keys.env")

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful


In [2]:
# 문서 로드 함수
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

# 문서 로드
raw_documents = load_jsonl('../dataset/documents.jsonl')

# 문서에서 docid를 포함한 Document 리스트 생성
documents = []
for doc in raw_documents:
    doc_id = doc['docid']  # JSONL 파일에서 docid 추출
    content = doc['content']  # 문서 내용 추출
    documents.append(Document(page_content=content, metadata={"doc_id": doc_id}))

In [3]:
# KIWI로 토큰화 함수 정의
kiwi = Kiwi()
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [4]:
# BM25와 FAISS 기반 검색기 생성
bm25 = BM25Retriever.from_documents(documents)
kiwi_bm25 = BM25Retriever.from_documents(documents, preprocess_func=kiwi_tokenize)

# FAISS 벡터 DB 생성 및 메타데이터와 함께 저장
faiss = FAISS.from_documents(documents, OpenAIEmbeddings()).as_retriever()

bm25_faiss_73 = EnsembleRetriever(
    retrievers=[bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
bm25_faiss_37 = EnsembleRetriever(
    retrievers=[bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)
kiwibm25_faiss_73 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.7, 0.3],
    search_type="mmr",
)
kiwibm25_faiss_37 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)

In [5]:
retrievers = {
    "bm25": bm25,
    "kiwi_bm25": kiwi_bm25,
    "faiss": faiss,
    "bm25_faiss_73": bm25_faiss_73,
    "bm25_faiss_37": bm25_faiss_37,
    "kiwi_bm25_faiss_73": kiwibm25_faiss_73,
    "kiwi_bm25_faiss_37": kiwibm25_faiss_37,
}

In [8]:
# 검색 결과 출력 함수
def print_search_results(retrievers, query):
    print(f"Query : {query}")
    for name, retriever in retrievers.items():
        results = retriever.invoke(query)
        print(f"{name} 결과:")
        for result in results:
            print(f"문서 ID: {result.metadata['doc_id']} 내용: {result.page_content}")
        print("===" * 20)

In [9]:
print(print_search_results(retrievers, "ktx의 평균 속도는 얼마인가?"))

Query : ktx의 평균 속도는 얼마인가?
bm25 결과:
문서 ID: e8d07278-c590-43e2-b99a-47d5adaf123d 내용: 아르투로는 3,000미터 경주에서 10분이라는 기록으로 결승점에 도달했습니다. 이를 통해 우리는 아르투로의 평균 속도를 계산할 수 있습니다. 평균 속도는 이동한 거리를 이동에 걸린 시간으로 나눈 값입니다. 따라서, 아르투로의 평균 속도는 3,000미터를 10분으로 나눈 값인 300미터/분입니다. 이를 미터/초로 변환하면 5 m/s가 됩니다. 따라서, 아르투로의 평균 속도는 5 m/s입니다.
문서 ID: 8ca2ebf5-9f0d-4d2f-bfef-92b44c920d2e 내용: 고무 오리 경주에서 우승한 오리는 10분 동안 300미터를 이동했습니다. 이를 통해 우승한 오리의 평균 속도를 계산할 수 있습니다. 평균 속도는 이동한 거리를 이동에 소요된 시간으로 나눈 값입니다. 따라서, 이 경우 오리의 평균 속도는 300미터를 10분으로 나눈 값인 30미터/분입니다. 그러나, 문제에서 속도를 미터/초로 표현하라고 하였으므로, 분 단위를 초 단위로 변환해야 합니다. 1분은 60초이므로, 30미터/분은 0.5미터/초입니다. 따라서, 고무 오리 경주에서 우승한 오리의 평균 속도는 0.5미터/초입니다.
문서 ID: 525f7b96-8422-4e57-b54b-32aff1d42aed 내용: 학생들은 학교 운동장에서 거북이를 관찰하였습니다. 그들은 거북이가 30분 동안 이동한 거리를 측정하였습니다. 거북이가 이 시간 동안 40미터를 걸었다면, 평균 속도는 얼마였을까요? 이 문제를 풀기 위해서는 거리를 시간으로 나누어야 합니다. 따라서, 거북이의 평균 속도는 40미터를 30분으로 나눈 값인 1.33m/min입니다. 그러나, 답안에서는 속도를 시간당 단위로 표기하였습니다. 따라서, 거북이의 평균 속도는 1.33m/min을 시간당 단위로 변환한 80m/h입니다.
문서 ID: fe6a254b-0f86-4025-b867-b35d56273c8d 